In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
titanic = pd.read_csv("/kaggle/input/titanic-dataset/Titanic-Dataset.csv")

In [ ]:
print(f"Rows: {titanic.shape[0]}")
print(f"Columns: {titanic.shape[1]}")

In [ ]:
titanic.columns

In [ ]:
titanic.head()

In [ ]:
titanic.describe()

In [ ]:
titanic.info()

In [ ]:
titanic.isna().sum()

Going to drop the Cabin column because there are lots of missing values
<br> 
Age and Embarked will be imputated

In [ ]:
titanic.nunique()

In [ ]:
titanic.columns

# Numerical Data

In [ ]:
titanic_num = titanic.select_dtypes(include="number")

In [ ]:
num_corr = titanic_num.corr()

plt.figure(figsize=(8,5))
sns.heatmap(num_corr, annot=True)
plt.show()

There is a moderate correlation between Pclass and Survived column target

In [ ]:
titanic_num.head()

In [ ]:
titanic_num.groupby("Survived").median()

In [ ]:
for col in titanic_num.columns[2:]:
    plt.hist(titanic_num[titanic_num.Survived==1][col], color="blue", alpha=.5, label="Survived")
    plt.hist(titanic_num[titanic_num.Survived==0][col], color="red", alpha=.5, label="Die")
    
    median_survived = titanic_num[titanic_num.Survived==1][col].mean()
    median_died = titanic_num[titanic_num.Survived==0][col].mean()
    
    # Draw vertical lines for medians
    plt.axvline(median_survived, color='blue', linestyle='solid', linewidth=1)
    plt.axvline(median_died, color='red', linestyle='dashed', linewidth=1)
    
    plt.title(col)
    plt.ylabel("Probability")
    plt.legend()
    plt.show()

Those who are in Pclass 3 have the highest death rate
Higher Fare have higher survival chance

In [ ]:
survived_num = titanic_num.loc[titanic_num["Survived"]==1, :]
died_num = titanic_num.loc[titanic_num["Survived"]==0, :]


sns.regplot(x=survived_num["Pclass"], y=survived_num["Fare"], color="blue")
sns.regplot(x=died_num["Pclass"], y=died_num["Fare"], color="red")
plt.show()

Higher Fare have higher Pclass

# Categorical Data

In [ ]:
titanic_s = titanic[["Name", "Sex", "Ticket", "Cabin", "Embarked", "Survived"]]
titanic_s

In [ ]:
survived_s = titanic_s[titanic_s["Survived"]==1]["Sex"].value_counts()
died_s = titanic_s[titanic_s["Survived"]==0]["Sex"].value_counts()

plt.bar(died_s.index, died_s, color="red", alpha=.5, label="Die")
plt.bar(survived_s.index, survived_s, color="blue", alpha=.5, label="Survived")
plt.title("Sex")
plt.ylabel("Probability")
plt.legend()
plt.show()

Females have higher survival rate than men

In [ ]:
survived_e = titanic_s[titanic_s["Survived"]==1]["Embarked"].value_counts()
died_e = titanic_s[titanic_s["Survived"]==0]["Embarked"].value_counts()
    
    
    
    
plt.bar(survived_e.index, survived_e, color="blue", alpha=.5, label="Survived")
plt.bar(died_e.index, died_e, color="red", alpha=.5, label="Died")
plt.title("Embarked")
plt.ylabel("Probability")
plt.legend()
plt.show()

Those who embarked at C have the highest survival rate

In [ ]:
titanic_s[titanic_s["Name"].str.contains(r"\(.*\)")].head(10)

In [ ]:
titanic_s

# Feature Engineering

## Categorical Values

In [ ]:
titanic_s["Title"] = titanic_s["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)

In [ ]:
titanic_s.Title.unique()

In [ ]:
survived_t = titanic_s[titanic_s["Survived"]==1]["Title"].value_counts()
died_t = titanic_s[titanic_s["Survived"]==0]["Title"].value_counts()
    
    
    
    
plt.bar(survived_t.index, survived_t, color="blue", alpha=.5, label="Survived")
plt.bar(died_t.index, died_t, color="red", alpha=.5, label="Died")
plt.title("Title")
plt.ylabel("Probability")
plt.xticks(rotation="vertical")
plt.legend()
plt.show()

People with Mr as their title have the highest death rate

## Numerical Values

In [ ]:
titanic_num["FamSize"] = titanic_num["Parch"] + titanic_num["SibSp"] + 1

In [ ]:
titanic_num["Fare_Pclass"] = titanic_num["Fare"] * titanic_num["Pclass"]

In [ ]:
sns.heatmap(titanic_num.corr(), annot=True)
plt.show()